### Libraries and Modules
[ORCA Library for Python](https://github.com/sybrenstuvel/Python-RVO2)

[RVO2 Library Documentation](https://gamma.cs.unc.edu/RVO2/documentation/2.0/)

In [1]:
import rvo2
import math
from csv import writer
import time
import numpy as np
import import_ipynb
import machines
import utils

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from machines.ipynb


### Global Parameters

In [7]:
time_step = 1
max_speed = 0.1
radius = 0.1
time_horizon = 20
time_horizon_obst = 0
velocity = (0.1, 0.1)

### Main Loop

In [18]:
def main_module(num_robots, seed, csv_writer):
    start_time = time.time()
    #ORCA Parameters initialization
    max_neighbors = num_robots-1
    neighbor_dist = math.sqrt(2*num_robots)
    agents = []
    #Robot class initialization
    robots = [machines.Robot(i) for i in range(num_robots)]
    utils.pos_gen(num_robots, robots, seed)
    init_pos  = np.array([agent.pos_i for agent in robots])
    final_pos = np.array([agent.pos_f for agent in robots])
    dist = np.array([agent.dist_total for agent in robots])
    #Environment initialization
    sim = rvo2.PyRVOSimulator(time_step, neighbor_dist, max_neighbors,
                              time_horizon, time_horizon_obst, radius,
                              max_speed)
    #Agent initialization
    for i in range(max_neighbors+1):
        agents.append(sim.addAgent(tuple(init_pos[i])))
        inst = final_pos[i] - init_pos[i]
        sim.setAgentPrefVelocity(agents[i], tuple(inst/np.linalg.norm(inst)))
    #Main Loop
    step = 0
    while not np.all([agent.inside for agent in robots]):
        sim.doStep()
        for i, agent_no in enumerate(agents):
            robots[i].pos_i = sim.getAgentPosition(agent_no)
            traveled_dist = np.linalg.norm(np.array(robots[i].pos_i)
                                               - init_pos[i])
            if traveled_dist > dist[i] and not robots[i].inside:
                robots[i].inside = True
        step += 1
    csv_writer.writerow([num_robots, seed, step, time.time() - start_time])

### Experiments

In [19]:
with open("/home/leduin/Escritorio/Tesis/Resultados/orca.csv","a") as file:
    csv_writer = writer(file)
    for i in range(1):
        num_robots = 5+i
        for seed in range(30):
            main_module(num_robots,seed,csv_writer)